In [ ]:


import pandas as pd
pd.options.display.max_columns=1000
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# link = 'https://drive.google.com/open?id=1DPZZQ43w8brRhbEMolgLqOWKbZbE-IQu' # The shareable link
# get id from the google drive files .csv after running and saing bigquery into google drive

##importing relevant files that were saved from Google BigQuery
# add country link here
# =pd.read_csv(io.BytesIO(uploaded["county_day_history.csv"])) 
id = '1kSXoAiXTtT_n4YINSX8qKzUMxQsqGq3o'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
country_day = pd.read_csv('Filename.csv')


# add covid search symptoms here
id = '1GTzGK5X9Cm_aBCyigPJDnjBBdsFMDNpP'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename2.csv')  
search_symptoms = pd.read_csv('Filename2.csv')
# search_symptoms=pd.read_csv("Google_Search_Symptoms_and_COVID_Cases_Counties_Daily.csv") 


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (428,429) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error #, mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_val_score

from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split

Data Exploration


In [ ]:
country_day.head()

,county_fips_code,country,date,min_temperature_air_2m_f,avg_temperature_air_2m_f,max_temperature_air_2m_f,min_humidity_relative_2m_pct,avg_humidity_relative_2m_pct,max_humidity_relative_2m_pct,min_humidity_specific_2m_gpkg,avg_humidity_specific_2m_gpkg,max_humidity_specific_2m_gpkg
0,49017,US,2020-04-10,33.4,46.4,58.4,15,31,51,2.0,2.5,3.0
1,29229,US,2020-04-10,31.0,45.4,58.9,25,40,61,2.0,2.6,3.0
2,19077,US,2020-04-10,23.3,39.9,53.2,24,49,84,2.0,2.4,3.0
3,20197,US,2020-04-10,25.3,44.8,58.6,24,40,69,2.0,2.4,3.0
4,29065,US,2020-04-10,28.6,44.7,59.9,19,39,64,2.0,2.4,3.0


In [ ]:
search_symptoms.head()

,sub_region_1,sub_region_2,sub_region_2_code,date,symptom_Abdominal_obesity,symptom_Abdominal_pain,symptom_Acne,symptom_Actinic_keratosis,symptom_Acute_bronchitis,symptom_Adrenal_crisis,symptom_Ageusia,symptom_Alcoholism,symptom_Allergic_conjunctivitis,symptom_Allergy,symptom_Amblyopia,symptom_Amenorrhea,symptom_Amnesia,symptom_Anal_fissure,symptom_Anaphylaxis,symptom_Anemia,symptom_Angina_pectoris,symptom_Angioedema,symptom_Angular_cheilitis,symptom_Anosmia,symptom_Anxiety,symptom_Aphasia,symptom_Aphonia,symptom_Apnea,symptom_Arthralgia,symptom_Arthritis,symptom_Ascites,symptom_Asperger_syndrome,symptom_Asphyxia,symptom_Asthma,symptom_Astigmatism,symptom_Ataxia,symptom_Atheroma,symptom_Attention_deficit_hyperactivity_disorder,symptom_Auditory_hallucination,symptom_Autoimmune_disease,symptom_Avoidant_personality_disorder,symptom_Back_pain,symptom_Bacterial_vaginosis,symptom_Balance_disorder,symptom_Beaus_lines,symptom_Bells_palsy,symptom_Biliary_colic,symptom_Binge_eating,symptom_Bleeding,symptom_Bleeding_on_probing,symptom_Blepharospasm,symptom_Bloating,symptom_Blood_in_stool,symptom_Blurred_vision,symptom_Blushing,symptom_Boil,symptom_Bone_fracture,symptom_Bone_tumor,symptom_Bowel_obstruction,symptom_Bradycardia,symptom_Braxton_Hicks_contractions,symptom_Breakthrough_bleeding,symptom_Breast_pain,symptom_Bronchitis,symptom_Bruise,symptom_Bruxism,symptom_Bunion,symptom_Burn,symptom_Burning_Chest_Pain,symptom_Burning_mouth_syndrome,symptom_Candidiasis,symptom_Canker_sore,symptom_Cardiac_arrest,symptom_Carpal_tunnel_syndrome,symptom_Cataplexy,symptom_Cataract,symptom_Chancre,symptom_Cheilitis,symptom_Chest_pain,symptom_Chills,symptom_Chorea,symptom_Chronic_pain,symptom_Cirrhosis,symptom_Cleft_lip_and_cleft_palate,symptom_Clouding_of_consciousness,symptom_Cluster_headache,symptom_Colitis,symptom_Coma,symptom_Common_cold,symptom_Compulsive_behavior,symptom_Compulsive_hoarding,symptom_Confusion,symptom_Congenital_heart_defect,symptom_Conjunctivitis,symptom_Constipation,symptom_Convulsion,symptom_Cough,symptom_Crackles,symptom_Cramp,symptom_Crepitus,symptom_Croup,symptom_Cyanosis,symptom_Dandruff,symptom_Delayed_onset_muscle_soreness,symptom_Dementia,symptom_Dentin_hypersensitivity,symptom_Depersonalization,symptom_Depression,symptom_Dermatitis,symptom_Desquamation,symptom_Developmental_disability,symptom_Diabetes,symptom_Diabetic_ketoacidosis,symptom_Diarrhea,symptom_Dizziness,symptom_Dry_eye_syndrome,symptom_Dysautonomia,symptom_Dysgeusia,symptom_Dysmenorrhea,symptom_Dyspareunia,symptom_Dysphagia,symptom_Dysphoria,symptom_Dystonia,symptom_Dysuria,symptom_Ear_pain,symptom_Eczema,symptom_Edema,symptom_Encephalitis,symptom_Encephalopathy,symptom_Epidermoid_cyst,symptom_Epilepsy,symptom_Epiphora,symptom_Erectile_dysfunction,symptom_Erythema,symptom_Erythema_chronicum_migrans,symptom_Esophagitis,symptom_Excessive_daytime_sleepiness,symptom_Eye_pain,symptom_Eye_strain,symptom_Facial_nerve_paralysis,symptom_Facial_swelling,symptom_Fasciculation,symptom_Fatigue,symptom_Fatty_liver_disease,symptom_Fecal_incontinence,symptom_Fever,symptom_Fibrillation,symptom_Fibrocystic_breast_changes,symptom_Fibromyalgia,symptom_Flatulence,symptom_Floater,symptom_Focal_seizure,symptom_Folate_deficiency,symptom_Food_craving,symptom_Food_intolerance,symptom_Frequent_urination,symptom_Gastroesophageal_reflux_disease,symptom_Gastroparesis,symptom_Generalized_anxiety_disorder,symptom_Generalized_tonic_clonic_seizure,symptom_Genital_wart,symptom_Gingival_recession,symptom_Gingivitis,symptom_Globus_pharyngis,symptom_Goitre,symptom_Gout,symptom_Grandiosity,symptom_Granuloma,symptom_Guilt,symptom_Hair_loss,symptom_Halitosis,symptom_Hay_fever,symptom_Headache,symptom_Heart_arrhythmia,symptom_Heart_murmur,symptom_Heartburn,symptom_Hematochezia,symptom_Hematoma,symptom_Hematuria,symptom_Hemolysis,symptom_Hemoptysis,symptom_Hemorrhoids,symptom_Hepatic_encephalopathy,symptom_Hepatitis,symptom_Hepatotoxicity,symptom_Hiccup,symptom_Hip_pain,symptom_Hives,symptom_Hot_flas

In [ ]:
merged_df=country_day.merge(search_symptoms, right_on=["sub_region_2_code","date"], left_on=["county_fips_code","date"], how="inner")

In [ ]:
merged_df.head()

,county_fips_code,country,date,min_temperature_air_2m_f,avg_temperature_air_2m_f,max_temperature_air_2m_f,min_humidity_relative_2m_pct,avg_humidity_relative_2m_pct,max_humidity_relative_2m_pct,min_humidity_specific_2m_gpkg,avg_humidity_specific_2m_gpkg,max_humidity_specific_2m_gpkg,sub_region_1,sub_region_2,sub_region_2_code,symptom_Abdominal_obesity,symptom_Abdominal_pain,symptom_Acne,symptom_Actinic_keratosis,symptom_Acute_bronchitis,symptom_Adrenal_crisis,symptom_Ageusia,symptom_Alcoholism,symptom_Allergic_conjunctivitis,symptom_Allergy,symptom_Amblyopia,symptom_Amenorrhea,symptom_Amnesia,symptom_Anal_fissure,symptom_Anaphylaxis,symptom_Anemia,symptom_Angina_pectoris,symptom_Angioedema,symptom_Angular_cheilitis,symptom_Anosmia,symptom_Anxiety,symptom_Aphasia,symptom_Aphonia,symptom_Apnea,symptom_Arthralgia,symptom_Arthritis,symptom_Ascites,symptom_Asperger_syndrome,symptom_Asphyxia,symptom_Asthma,symptom_Astigmatism,symptom_Ataxia,symptom_Atheroma,symptom_Attention_deficit_hyperactivity_disorder,symptom_Auditory_hallucination,symptom_Autoimmune_disease,symptom_Avoidant_personality_disorder,symptom_Back_pain,symptom_Bacterial_vaginosis,symptom_Balance_disorder,symptom_Beaus_lines,symptom_Bells_palsy,symptom_Biliary_colic,symptom_Binge_eating,symptom_Bleeding,symptom_Bleeding_on_probing,symptom_Blepharospasm,symptom_Bloating,symptom_Blood_in_stool,symptom_Blurred_vision,symptom_Blushing,symptom_Boil,symptom_Bone_fracture,symptom_Bone_tumor,symptom_Bowel_obstruction,symptom_Bradycardia,symptom_Braxton_Hicks_contractions,symptom_Breakthrough_bleeding,symptom_Breast_pain,symptom_Bronchitis,symptom_Bruise,symptom_Bruxism,symptom_Bunion,symptom_Burn,symptom_Burning_Chest_Pain,symptom_Burning_mouth_syndrome,symptom_Candidiasis,symptom_Canker_sore,symptom_Cardiac_arrest,symptom_Carpal_tunnel_syndrome,symptom_Cataplexy,symptom_Cataract,symptom_Chancre,symptom_Cheilitis,symptom_Chest_pain,symptom_Chills,symptom_Chorea,symptom_Chronic_pain,symptom_Cirrhosis,symptom_Cleft_lip_and_cleft_palate,symptom_Clouding_of_consciousness,symptom_Cluster_headache,symptom_Colitis,symptom_Coma,symptom_Common_cold,symptom_Compulsive_behavior,symptom_Compulsive_hoarding,symptom_Confusion,symptom_Congenital_heart_defect,symptom_Conjunctivitis,symptom_Constipation,symptom_Convulsion,symptom_Cough,symptom_Crackles,symptom_Cramp,symptom_Crepitus,symptom_Croup,symptom_Cyanosis,symptom_Dandruff,symptom_Delayed_onset_muscle_soreness,symptom_Dementia,symptom_Dentin_hypersensitivity,symptom_Depersonalization,symptom_Depression,symptom_Dermatitis,symptom_Desquamation,symptom_Developmental_disability,symptom_Diabetes,symptom_Diabetic_ketoacidosis,symptom_Diarrhea,symptom_Dizziness,symptom_Dry_eye_syndrome,symptom_Dysautonomia,symptom_Dysgeusia,symptom_Dysmenorrhea,symptom_Dyspareunia,symptom_Dysphagia,symptom_Dysphoria,symptom_Dystonia,symptom_Dysuria,symptom_Ear_pain,symptom_Eczema,symptom_Edema,symptom_Encephalitis,symptom_Encephalopathy,symptom_Epidermoid_cyst,symptom_Epilepsy,symptom_Epiphora,symptom_Erectile_dysfunction,symptom_Erythema,symptom_Erythema_chronicum_migrans,symptom_Esophagitis,symptom_Excessive_daytime_sleepiness,symptom_Eye_pain,symptom_Eye_strain,symptom_Facial_nerve_paralysis,symptom_Facial_swelling,symptom_Fasciculation,symptom_Fatigue,symptom_Fatty_liver_disease,symptom_Fecal_incontinence,symptom_Fever,symptom_Fibrillation,symptom_Fibrocystic_breast_changes,symptom_Fibromyalgia,symptom_Flatulence,symptom_Floater,symptom_Focal_seizure,symptom_Folate_deficiency,symptom_Food_craving,symptom_Food_intolerance,symptom_Frequent_urination,symptom_Gastroesophageal_reflux_disease,symptom_Gastroparesis,symptom_Generalized_anxiety_disorder,symptom_Generalized_tonic_clonic_seizure,symptom_Genital_wart,symptom_Gingival_recession,symptom_Gingivitis,symptom_Globus_pharyngis,symptom_Goitre,symptom_Gout,symptom_Grandiosity,symptom_Granuloma,symptom_Guilt,symptom_Hair_loss,symptom_Halitosis,symptom_Hay_fever,symptom_Headache,symptom_Heart_arrhythmia,symptom_H

In [ ]:
merged_df_Cali = merged_df[merged_df.sub_region_1=='California']
merged_df_Cali.county_fips_code.unique()

array([6051, 6037, 6019, 6015, 6021, 6067, 6005, 6001, 6103, 6093, 6063,
       6039, 6043, 6053, 6065, 6033, 6081, 6095, 6009, 6107, 6105, 6069,
       6047, 6115, 6113, 6013, 6101, 6059, 6099, 6109, 6079, 6111, 6057,
       6071, 6045, 6097, 6075, 6041, 6025, 6023, 6077, 6089, 6049, 6027,
       6035, 6017, 6031, 6083, 6073, 6061, 6011, 6029, 6007, 6085, 6055,
       6087])

In [ ]:
import random

random.seed = 10
# random_FIPS=random.sample(list(merged_df_Cali.county_fips_code.unique()),10)
random_FIPS=[6089, 6077, 6079, 6011, 6057, 6097, 6093, 6035, 6067, 6025]

In [ ]:


from scipy.stats import pearsonr
import pandas as pd

fips_code = 6017
# merged_df_Cali[merged_df_Cali.county_fips_code == fips_code]
merged_df_Cali[merged_df_Cali.county_fips_code.isin([6057,6079])].fillna(0).groupby('county_fips_code')[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']].corr()
# 6057,6079

avg_temperature_air_2m_f  \
county_fips_code                                                          
6057             avg_temperature_air_2m_f                      1.000000   
                 avg_humidity_relative_2m_pct                 -0.709904   
                 symptom_Fever                                -0.466548   
6079             avg_temperature_air_2m_f                      1.000000   
                 avg_humidity_relative_2m_pct                 -0.748316   
                 symptom_Fever                                -0.398721   

                                               avg_humidity_relative_2m_pct  \
county_fips_code                                                              
6057             avg_temperature_air_2m_f                         -0.709904   
                 avg_humidity_relative_2m_pct                      1.000000   
                 symptom_Fever                                     0.420773   
6079             avg_temperature_air_2m_f                         -0.748316   
                 avg_humidity_relative_2m_pct                      1.000000   
                 symptom_Fever                                     0.417804   

                                               symptom_Fever  
county_fips_code                                              
6057             avg_temperature_air_2m_f          -0.466548  
                 avg_humidity_relative_2m_pct       0.420773  
                 symptom_Fever                      1.000000  
6079             avg_temperature_air_2m_f          -0.398721  
                 avg_humidity_relative_2m_pct       0.417804  
                 symptom_Fever                      1.000000

In [ ]:
fips_code = 6011
merged_df_Cali[merged_df_Cali.county_fips_code == fips_code][['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']]

,avg_temperature_air_2m_f,avg_humidity_relative_2m_pct,symptom_Fever
374,62.4,62,NaN
593,69.0,61,NaN
869,41.4,75,NaN
1323,74.2,32,NaN
1929,57.6,41,NaN
...,...,...,...
147607,79.1,52,NaN
147981,49.8,79,NaN
148386,57.5,45,NaN
148815,50.1,84,NaN


In [ ]:
merged_df_Cali[merged_df_Cali.county_fips_code.isin(random_FIPS)].groupby('county_fips_code')['symptom_Fever'].count()

county_fips_code
6011      0
6025    364
6035      0
6057    362
6067    364
6077    364
6079    364
6089    364
6093    288
6097    364
Name: symptom_Fever, dtype: int64

In [ ]:
# from sklearn import svm
# import numpy as np
# from sklearn.model_selection import train_test_split
# scores = []
# for fips_code in random_FIPS:
#   merged_df_FIPS = merged_df_Cali[merged_df_Cali.county_fips_code == fips_code]
#   merged_df_filt_symptom=merged_df_FIPS[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']].dropna()
#   if len(merged_df_filt_symptom) != 0:
    
      
#     # train_set, test_set= np.split(merged_df_filt_symptom, [int(.8 *len(merged_df_filt_symptom))])
#     X_train, X_test, y_train, y_test = train_test_split( merged_df_filt_symptom[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']], merged_df_filt_symptom[['symptom_Fever']], test_size=0.20, random_state=42, shuffle=True)
#     # X_train=train_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
#     # y_train=train_set[['symptom_Fever']].values
#     # y_train=y_train.ravel()
#     # X_test=test_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
#     # y_test=test_set[['symptom_Fever']].values

#     # y_test=y_test.ravel()
#     rf = RandomForestRegressor(n_estimators = 1000, random_state = 12)
#     # Train the model on training data

#     scores.append(rf.fit(X_train, y_train).score(X_test, y_test))


Running models

In [ ]:

scores = []
MSE = []
MAPE=[]
for fips_code in random_FIPS:
  merged_df_FIPS = merged_df_Cali[merged_df_Cali.county_fips_code == fips_code]
  merged_df_filt_symptom=merged_df_FIPS[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']].dropna()
  if len(merged_df_filt_symptom) != 0:
    
      
    # train_set, test_set= np.split(merged_df_filt_symptom, [int(.8 *len(merged_df_filt_symptom))])
    X_train, X_test, y_train, y_test = train_test_split( merged_df_filt_symptom[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values, merged_df_filt_symptom[['symptom_Fever']].values, test_size=0.25, random_state=42, shuffle=True)
    # X_train=train_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
    # y_train=train_set[['symptom_Fever']].values
    y_train=y_train.ravel()
    # X_test=test_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
    # y_test=test_set[['symptom_Fever']].values

    y_test=y_test.ravel()

    rf = RandomForestRegressor(n_estimators = 2000, random_state = 22)
    # clf = svm.SVR(kernel='rbf', C=1).fit(X_train, y_train)
    # Train the model on training data

    scores.append(rf.fit(X_train, y_train).score(X_test, y_test))
    
    MAPE.append(mean_squared_error(y_test,rf.predict(X_test)))

In [ ]:
scores, MAPE, random_FIPS

([-0.04739754148272146,
  0.0341597817761381,
  0.026938962974999536,
  0.4824300022976574,
  0.21446419301993971,
  0.13161711982562396,
  0.0449779245284575,
  0.08519202323776942],
 [6.243258737316097,
  4.665261324919151,
  6.6268497892476095,
  2.334328528410029,
  6.405513346739007,
  7.596798570171147,
  4.938716857128172,
  5.729250073138296],
 [6089, 6077, 6079, 6011, 6057, 6097, 6093, 6035, 6067, 6025])

In [ ]:
merged_df_FIPS.symptom_Fever.std()**2

5.194083903974813

In [ ]:
from sklearn import datasets, ensemble

merged_df_filt_symptom=merged_df_Cali[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']].fillna(0)

  
# train_set, test_set= np.split(merged_df_filt_symptom, [int(.8 *len(merged_df_filt_symptom))])
X_train, X_test, y_train, y_test = train_test_split( merged_df_filt_symptom[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values, merged_df_filt_symptom[['symptom_Fever']].values, test_size=0.20, random_state=24, shuffle=True)
# X_train=train_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
# y_train=train_set[['symptom_Fever']].values
y_train=y_train.ravel()
# X_test=test_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
# y_test=test_set[['symptom_Fever']].values

y_test=y_test.ravel()
rf = RandomForestRegressor(n_estimators = 1000, random_state = 24)

print(rf.fit(X_train, y_train).score(X_test, y_test))

print("error:",mean_squared_error(y_test,rf.predict(X_test)))
# clf = svm.SVR(kernel='rbf', C=1)
# # Train the model on training data
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print(mse)
# print(clf.fit(X_train, y_train).score(X_test, y_test))

-0.1654584151593126
error: 10.12587934140445
7.682074829886767


In [ ]:
reg.score(X_test, y_test)

0.1154511075405058

In [ ]:
from sklearn.metrics import mean_squared_error
merged_df_filt_symptom=merged_df[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']].dropna()

  
# train_set, test_set= np.split(merged_df_filt_symptom, [int(.8 *len(merged_df_filt_symptom))])
X_train, X_test, y_train, y_test = train_test_split( merged_df_filt_symptom[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values, merged_df_filt_symptom[['symptom_Fever']].values, test_size=0.20, random_state=12, shuffle=True)
# X_train=train_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
# y_train=train_set[['symptom_Fever']].values
y_train=y_train.ravel()
# X_test=test_set[['avg_temperature_air_2m_f','avg_humidity_relative_2m_pct']].values
# y_test=test_set[['symptom_Fever']].values

y_test=y_test.ravel()
rf = RandomForestRegressor(n_estimators = 100, random_state = 12)
print(rf.fit(X_train, y_train).score(X_test, y_test))
print("error:",mean_squared_error(y_test,rf.predict(X_test)))
clf = svm.SVR(kernel='rbf', C=1)
# Train the model on training data

print(clf.fit(X_train, y_train).score(X_test, y_test))

-0.1644552832371735
error: 6.964241359017473
-0.03278472954281786


In [ ]:

regr = svm.SVR()
regr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:

regr.score(X_test, y_test)

0.1968638703100204

In [ ]:
merged_df_filt_symptom

,avg_temperature_air_2m_f,avg_humidity_relative_2m_pct,symptom_Fever


In [ ]:
merged_df_Cali[merged_df_Cali.county_fips_code == fips_code].symptom_Fever

374      NaN
593      NaN
869      NaN
1323     NaN
1929     NaN
          ..
147607   NaN
147981   NaN
148386   NaN
148815   NaN
149050   NaN
Name: symptom_Fever, Length: 364, dtype: float64

In [ ]:
merged_df_Cali[['county_fips_code','avg_temperature_air_2m_f','avg_humidity_relative_2m_pct','symptom_Fever']]

,county_fips_code,avg_temperature_air_2m_f,avg_humidity_relative_2m_pct,symptom_Fever
12,6051,37.6,63,NaN
20,6037,39.5,92,5.60
35,6019,56.9,69,6.24
40,6015,54.6,81,6.97
46,6021,61.5,63,NaN
...,...,...,...,...
149390,6089,32.7,100,16.00
149391,6081,39.6,95,21.26
149400,6061,34.7,96,17.25
149402,6041,44.3,87,23.04
